In [1]:
import pickle
import json
import sys
import os
import string
import requests
import ast  # for string to list: ast.literal_eval()
from bs4 import BeautifulSoup
import bs4
import time
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import date
import pandas as pd
import numpy as np
import re
from tqdm.notebook import tqdm
import warnings
data_dir = "/Users/vigadam/Documents/github/media_network/media_data/html_pages/mno/"

In [78]:
class Helper:
    import json

    from bs4 import BeautifulSoup
    import bs4

    import pandas as pd
    import numpy as np

    import re

    import warnings

    warnings.filterwarnings("ignore")

    def __init__(self, df):
        None

    def title_main(self, element, _id):
        for i, soup in tqdm(enumerate(self.soups)):
            try:
                df.loc[self.soups.index[i], "Title"] = (
                    BeautifulSoup(soup).find(attrs={element: _id}).get_text().strip()
                )
            except AttributeError:
                df.loc[self.soups.index[i], "Title"] = None

    def link(self):
        for i, soup in tqdm(enumerate(self.soups)):
            soup_for_links = BeautifulSoup(soup).find_all("a")
            link_list = []
            for item in soup_for_links:
                if type(item.get("href")) == str:
                    link_list.append(item.get("href"))
            df.loc[self.soups.index[i], "links"] = str(list(set(link_list)))

    def text_main(self, element, class_id):
        for i, soup in tqdm(enumerate(self.soups)):
            try:
                soup_for_text = (
                    BeautifulSoup(soup)
                    .find(element, class_=re.compile(class_id))
                    .find_all("p")
                )
                text_list = []
                for item in soup_for_text:
                    if soup_for_text != "":
                        text_list.append(item.text)
                df.loc[self.soups.index[i], "Text"] = " ".join(text_list)
            except AttributeError:
                df.loc[self.soups.index[i], "Text"] = None

    def author_main(self, element, _id):
        for i, soup in tqdm(enumerate(self.soups)):
            try:
                df.loc[self.soups.index[i], "Author"] = (
                    BeautifulSoup(soup).find(attrs={element: _id}).get_text().strip()
                )
            except:
                df.loc[self.soups.index[i], "Author"] = None

    def category_main(self, element, id_):
        for i, soup in tqdm(enumerate(self.soups)):
            try:
                df.loc[self.soups.index[i], "Category"] = (
                    BeautifulSoup(soup).find(attrs={element: id_}).text.strip()
                )

            except:
                df.loc[self.soups.index[i], "Category"] = None

    def tags_main(self, element, id_):
        for i, soup in tqdm(enumerate(self.soups)):
            try:
                df.loc[self.soups.index[i], "Tags"] = (
                    BeautifulSoup(soup).find(attrs={element: id_}).text.strip()
                )

            except:
                df.loc[self.soups.index[i], "Tags"] = None
        df["Tags"] = df["Tags"].str.split("\n\n\n")

    def clean(self):
        self.title()
        self.text()
        self.link()
        self.author()
        self.source()
        self.tags()

In [79]:
class Mno(Helper):
    def __init__(self, df):
        self.data = df
        self.soups = self.data["soup"].dropna()

    def title(self):
        Helper.title_main(self, "class", "et_main_title")

    def text(self):
        for i, soup in tqdm(enumerate(self.soups)):
            try:
                textlist = (
                    BeautifulSoup(soup)
                    .find(attrs={"class": "entry-content clearfix"})
                    .find_all("p")
                )

                df.loc[self.soups.index[i], "Text"] = (
                    " ".join([item.get_text().strip() for item in textlist])
                    .replace("\xa0", "")
                    .strip()
                )
                date_text = (
                    BeautifulSoup(soup)
                    .find(attrs={"class": "en-article-dates-main"})
                    .get_text()
                    .split(" ")
                )

                df.loc[self.soups.index[i], "year"] = date_text[0].replace(".", "")
                df.loc[self.soups.index[i], "month"] = date_text[1]
                df.loc[self.soups.index[i], "day"] = date_text[2].replace(".", "")

            except AttributeError:
                df.loc[self.soups.index[i], "Text"] = None
                df.loc[self.soups.index[i], "year"] = None
                df.loc[self.soups.index[i], "month"] = None
                df.loc[self.soups.index[i], "day"] = None

    def author(self):
        Helper.author_main(self, "class", "en-article-author")

    def source(self):
        for i, soup in tqdm(enumerate(self.soups)):
            try:
                df.loc[self.soups.index[i], "Source"] = (
                    BeautifulSoup(soup)
                    .find(attrs={"class": "en-article-source col-sm"})
                    .get_text()
                    .strip()
                )

            except:
                df.loc[self.soups.index[i], "Source"] = None

    def tags(self):
        for i, soup in tqdm(enumerate(self.soups)):
            try:
                df.loc[self.soups.index[i], "Tags"] = str(
                    [
                        item.get_text()
                        for item in BeautifulSoup(soup)
                        .find(attrs={"class": "en-article-tags"})
                        .find_all("a")
                    ]
                )

            except:
                df.loc[self.soups.index[i], "Tags"] = None

In [80]:
data_files = os.listdir(data_dir + "/2020")

data_files = [data_dir + "/2020/" + item for item in data_files]

In [81]:
df_clean = pd.DataFrame()
for data_file in tqdm(data_files):

    df = pd.read_pickle(data_file)

    Mno(df).clean()

    df_clean = pd.concat([df_clean, df.drop(columns=["Unnamed: 0", "soup"])])

KeyboardInterrupt: 

In [2]:
clean_data_dir = "/Users/vigadam/Dropbox/My Mac (MacBook-Air.local)/Documents/github/media_network/media_data/clean_text/pickle_format"

In [3]:
df = pd.read_pickle(clean_data_dir +"/2020/mno_clean.pkl")

In [13]:
months = {
    "december": 12,
    "november": 11,
    "október": 10,
    "szeptember": 9,
    "augusztus": 8,
    "július": 7,
    "június": 6,
    "május": 5,
    "április": 4,
    "március": 3,
    "február": 2,
    "január": 1,
}

In [20]:
df["month"] = df["month"].map(months)

In [25]:
df.to_pickle(
    "/Users/vigadam/Dropbox/My Mac (MacBook-Air.local)/Documents/github/media_network/media_data/clean_text/pickle_format/2020/mno_clean.pkl"
)

In [26]:
df.to_csv(
    "/Users/vigadam/Dropbox/My Mac (MacBook-Air.local)/Documents/github/media_network/media_data/clean_text/csv_format/2020/mno_clean.csv"
)

In [12]:
df["rovat"].value_counts().head(10)

belfold         13379
kulfold         10078
gazdasag         6343
sport            4720
kultura          2939
mozaik           1556
lugas-rovat      1163
velemeny          897
orszagszerte      666
sztarvilag        420
Name: rovat, dtype: int64